# 7. Plotting in Python

1. [introduction](#intro)
1. [Introduction to Bokeh](#intro bokeh)
1. [interactive plotting](#interactive)
1. [matplotlib](#matplotlib)

## 7.1 introduction <a name="intro"/>

There are many plotting libraries, depending on your application. Here are some of the 

* [matplotlib](http://matplotlib.org): matlab-like, no "one and only one obvious way &#8230;"

* [seaborn](http://seaborn.pydata.org): matplotlib overlay, statistical data visualisation

* [mayavi](http://code.enthought.com/projects/mayavi/): 3D rendering by [Enthought](https://www.enthought.com)
<!-- .element: class="fragment fade-out" data-fragment-index="1" -->

* [bokeh](https://bokeh.pydata.org/en/latest/index.html): javascript = notebook + interaction, interfaces with `python`, `R`, `scala`, and `julia`. Maintained by Anaconda inc.

* [plot.ly](https://plot.ly/python/): recently offered for free to the community; interactive plots; more features than Bokeh; separating out layout and data (as in web CSS vs. HTML); interfaces with Matlab, Python, Julia, &hellip;


## 7.2 Introduction to Bokeh <a name="intro bokeh" />

For the `zip` command on line 13, see also [zip and unzip](https://docs.python.org/3/library/functions.html#zip)

* Keyboard shortcuts &rightarrow; <kbd>ESC</kbd>, then <kbd>H</kbd>
* Display line numbers &rightarrow; <kbd>ESC</kbd>, then <kbd>L</kbd>

In [ ]:
from bokeh.plotting import figure, gridplot, show
from bokeh.io import output_notebook
import math
from bokeh.models import ColumnDataSource

# integrate into notebook
output_notebook()

# prepare some date
N = 300
x = [x * 4 * math.pi / (N - 1) for x in range(N)]
y = [(math.sin(t), math.cos(t), math.cos(t) + math.sin(t)) for t in x] 
y0, y1, y2 = zip(*y)  

# create a new plot
s1 = figure(width=300, plot_height=300, title="cos", y_axis_label="y", x_axis_label="x")
s1.circle(x, y0, size=10, color="navy", alpha=0.5)

# NEW: create a new plot and share both ranges
s2 = figure(width=300, height=300, x_range=s1.x_range, y_range=s1.y_range, title="sin", x_axis_label="x")
s2.triangle(x, y1, size=10, color="firebrick", alpha=0.5)

# NEW: create a new plot and share only one range
s3 = figure(width=300, height=300, x_range=s1.x_range, title="cos+sin", x_axis_label="x")
s3.square(x, y2, size=10, color="olive", alpha=0.5)

# NEW: put the subplots in a gridplot
p = gridplot([[s1, s2, s3]], toolbar_location=None)

# show the results
show(p)

### 7.2.1 First Bokeh plot

1. A circle plot where the power of lists is illustrated (look at the size keyword argument)
1. Make a figure with a line glyph `p.line(x, y)`

In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from random import randint, uniform

output_notebook()

N = 20

p = figure(title="My first Bokeh plot", x_axis_label="pos x")
p.circle([uniform(-2, 2) for _ in range(N)], [uniform(-2, 2) for _ in range(N)],
         size=[randint(2, 5)*5 for _ in range(N)],
         color='navy', alpha=0.3)
show(p)

In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_file
from random import randint, uniform

output_file("myfirstBokehplot.html")

N = 20

p = figure(title="My first Bokeh plot")
p.circle([uniform(-2, 2) for _ in range(N)], [uniform(-2, 2) for _ in range(N)],
         size=[randint(2, 5)*5 for _ in range(N)],
         color='navy', alpha=0.3)
show(p) # will open a new tab in the browser and save to myfirstBokehplot.html

In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from math import exp

output_notebook()

N = 51
x = range(N)
x = [2 * (z/(N-1) - .5) for z in x]
y = [exp(-z ** 2 * 8) for z in x]

p = figure(title="A line plot")
p.line(x, y, line_width=2)
p.circle(x, y, fill_color="white", size=8, line_width=2)
show(p)

## 7.3 Interactive plots <a name="interactive" />

### 7.3.1 An introduction to ipywidgets and interact

Documentation of the [Jupyter ipywidgets](http://ipywidgets.readthedocs.io/en/stable/index.html). Look at how the interact function automatically chooses the interactive [widget](http://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html) as a function of your data type:

| Keyword argument | Widget |
|:--|:---|
| `True` or `False` | Checkbox |
| 'Hi there' | Text |
| value or (min,max) or (min,max,step) if integers are passed | IntSlider |
| value or (min,max) or (min,max,step) if floats are passed | FloatSlider |
| ['orange','apple'] or {'one':1,'two':2} | Dropdown |

In [ ]:
from ipywidgets import interact

def m(x, y):
    return (x, y)

interact(m, x=True, y=(1.01)) # using interact as a function modifier

In [ ]:
@interact(x=True, y=1.0)   # using interact as a decorator
def g(x, y):
    return (x, y)

### 7.3.2 Combining interact with Bokeh

First load some modules

In [ ]:
from ipywidgets import interact

from bokeh.io import output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

from math import pi, sin   # a math constant and function

output_notebook()          # show and push_notebook should send data to notebook

Set up data and basic plot

In [ ]:
# Set up data
N = 200
x = [2 * pi * t / (N-1) for t in range(N)]
y = [sin(t) for t in x]

# Set up figure
p = figure(title="a simple interactive plotting example", plot_height=300, plot_width=800, y_range=(-2.5, 2.5))
r = p.line(x, y, line_width=2)  # need to attribute this to a variable/handle for later reference
h = show(p, notebook_handle=True)

And now let the interaction come into play

In [ ]:
print('Interact with {}'.format(h.doc.title))
@interact(k=(.1, 10, .1), phi=(0., 2*pi, .1), a=(0., 2, .1), b=(-1., 1, .1))
def update(k=1, phi=0, a=1, b=0):
    r.data_source.data['y'] = [a * sin(k * t + phi) + b for t in x]
    push_notebook(handle=h)

## 7.4 Matplotlib <a name="matplotlib" />
### 7.4.1 Basic functionalities of the matplotlib package (using `pyplot`)

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact
# you could choose the "%matplotlib inline" magic as well below, but not when interacting
%matplotlib notebook

In [ ]:
fig = plt.figure(figsize=(9.5, 4.75)) # prepare figure, mysterious units
ax = fig.subplots(nrows=1, ncols=2, sharey=True)         # prepare axis
ax[0].plot(range(10), [x * 2 for x in [1, 8, 3, 2, 1, 9, -1, 3, 2, -2]], label="data") # labels for later insertion into legend
ax[0].plot(range(10), [1]*10, label="constant", linestyle="--")
ax[0].set_xlim((0, 9))

#Set y-ticks of subplot 2 invisible
plt.setp(ax[1].get_yticklabels(), visible=False)
ax[1].plot(range(10), [1, 8, 3, 2, 1, 9, -1, 3, 2, -2][::-1], label="data") # labels for later insertion into legend
ax[1].plot(range(10), [1]*10, label="constant", linestyle="--")
ax[1].legend()
ax[1].set_xlim((0, 9))

#Adjust the widths between the subplots
plt.subplots_adjust(wspace = -.059)

### 7.4.2 And adding interaction

In [ ]:
# Adding some interaction
@interact(translate=(-3, 3, 1), zoom=(3, 9, 2))
def update_plots(translate, zoom):
    z = (zoom - 1) / 2
    ax[0].set_xlim((4 + translate-z, 4 + translate+z))
    ax[1].set_xlim((4 + translate-z, 4 + translate+z))
    # fig.canvas.draw_idle()